# FNOL (First Notice of Loss) Processing

## Overview

This notebook processes First Notice of Loss (FNOL) data for insurance claims. It extracts, validates, and structures FNOL data with completeness checks to prepare it for downstream processing by fraud detection and policy verification teams.

## Workflow

1. **Import Dependencies** - Load required libraries (Strands, AWS Bedrock, JSON)
2. **AWS Configuration** - Configure Bedrock client and model
3. **Load FNOL Data** - Read and parse FNOL.json file
4. **FNOL Processing Agent** - Extract, validate, and structure claim data
5. **Data Quality Checks** - Flag missing fields and inconsistencies
6. **Output Structured Data** - Generate validated JSON for next stages


## 1. Install Dependencies

In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.40.50
botocore==1.40.76
opentelemetry-instrumentation-boto==0.60b1
bedrock-agentcore==0.1.7
bedrock-agentcore-starter-toolkit==0.1.24


In [2]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

In [3]:
![ ! -d "agents" ] && mkdir agents

## Create FNOL Processing Agent

This agent acts as a Claims Data Extraction Specialist that:
- Extracts key data points from FNOL documents
- Validates data completeness and consistency
- Flags potential data quality issues
- Generates structured JSON output for downstream processing

In [4]:
%%writefile agents/claimvalidation-agent.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
from strands_tools import retrieve

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException



# Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Add a handler to see the logs
# logging.basicConfig(
#     format="%(levelname)s | %(name)s | %(message)s", 
#     handlers=[logging.StreamHandler()]
# )

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used




# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")

# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:161615149547:automated-reasoning-policy/malxiyr0ojy2"
GUARDRAIL_ID = "an852wptcjol"
GUARDRAIL_VERSION = "4"
KNOWLEDGE_BASE_ID = "CZDJXI9C4E"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
# MODEL_ID = "us.amazon.nova-lite-v1:0"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID



# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        print ("HERE LOOKIE HERE",content_to_validate)
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = contentbedrock_model
            new_output = new_output + f"\n\nfindings: {self.findings}"
            new_output = new_output + f"\n\nclaim_valid: {self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""


# Create agent with the guardrail-protected model
agent = Agent(
    name="Claims Validator - ARC",
    description="A Single agent with Claims Validation tools capabilities",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve],
    system_prompt=agent_instructions
)



################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/claimvalidation-agent.py


In [24]:
%%writefile agents/claimvalidation-agent-2.py

import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn

from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
from strands_tools import retrieve

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3
from typing import Dict, Any, List
from dataclasses import dataclass

from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException



print("✓ All dependencies imported successfully")



# Configure the root strands logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used






# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")




# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:161615149547:automated-reasoning-policy/malxiyr0ojy2"
GUARDRAIL_ID = "an852wptcjol"
GUARDRAIL_VERSION = "4"
KNOWLEDGE_BASE_ID = "CZDJXI9C4E"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
# MODEL_ID = "us.amazon.nova-lite-v1:0"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID



# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        print ("HERE LOOKIE HERE",content_to_validate)
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = contentbedrock_model
            new_output = new_output + f"\n\nfindings: {self.findings}"
            new_output = new_output + f"\n\nclaim_valid: {self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str




# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)


# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    # model_id=MODEL_ID,
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""


# Create agent with the guardrail-protected model
agent = Agent(
    name="Claims Validator - ARC",
    description="A Single agent with Claims Validation tools capabilities",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve],
    system_prompt=agent_instructions
)


print("Claims Validation Processing Agent created")



################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/claimvalidation-agent-2.py


In [6]:
%%writefile agents/fnol-agent-2.py

import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

from typing import Dict, Any, List
from dataclasses import dataclass



print("✓ All dependencies imported successfully")




logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")



# Initialize Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
)



# Create FNOL Processing Agent
fnol_agent = Agent(
    name="FNOL Data Extraction Specialist",
    description="Intelligent agent for automated auto insurance claims FNOL processing",
    model=model,
    system_prompt="""You are a Claims Data Extraction Specialist.

ROLE: Process First Notice of Loss (FNOL) forms and extract structured claim data.

INPUTS: Raw FNOL document/form data
OUTPUTS: Structured JSON with validated claim information

INSTRUCTIONS:
1. Extract key data points:
   - Policy number, claim date, incident location, claimant details
   - Incident description, damages reported, witnesses
   - Supporting documentation references

2. Validate data completeness:
   - Flag missing required fields
   - Identify inconsistencies in dates/locations
   - Check format compliance (policy numbers, contact info)

3. Flag potential data quality issues with specific error codes
4. If critical information is missing, generate specific follow-up questions

ERROR HANDLING: If form is illegible or severely incomplete, flag for manual review with detailed reasoning.

QUALITY CHECK: Ensure all extracted monetary amounts, dates, and identifiers are properly formatted.

OUTPUT FORMAT: 
Return the same structured JSON object provided in the input

Return, in a separete section, the list recommendations: Next steps or follow-up actions needed
"""
)

print("FNOL Processing Agent created")



################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=fnol_agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/fnol-agent-2.py


In [25]:
%%writefile agents/requirements.txt
boto3==1.40.50
bedrock-agentcore==0.1.7
strands-agents[a2a]
strands-agents-tools
pyyaml
ddgs

Overwriting agents/requirements.txt


#### Setup Cognito User Pool

In [26]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_JkqGFEXTz
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_JkqGFEXTz/.well-known/openid-configuration
Client ID: ftjv7dm96qf84cc4n245krvt8
Bearer Token: eyJraWQiOiJsdnVjTXJjNmpqOXI2Yit2RlpnNks1WnZxMkF3ZjhPRklrXC9cLys5Zk8xUlU9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiIxNDI4MTQ2OC0xMDQxLTcwNWYtYmNhZi05MmZiMzNjY2EzMzUiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9Ka3FHRkVYVHoiLCJjbGllbnRfaWQiOiJmdGp2N2RtOTZxZjg0Y2M0bjI0NWtydnQ4Iiwib3JpZ2luX2p0aSI6IjM0ODI0MmQ0LWRhNTItNDIzYy05Zjc0LTZiYjlkYTcyODg5OCIsImV2ZW50X2lkIjoiZTIwZGQ1NGMtYTEyOC00MzMxLTkyNjAtNzZlNTgwYmNkMjc0IiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJhd3MuY29nbml0by5zaWduaW4udXNlci5hZG1pbiIsImF1dGhfdGltZSI6MTc2OTEzMDgxOSwiZXhwIjoxNzY5MTM0NDE5LCJpYXQiOjE3NjkxMzA4MTksImp0aSI6IjQ1ZDA1MzlkLWQyYjUtNDQwNi05NTc5LTMyZGExNDdiNDU4YyIsInVzZXJuYW1lIjoidGVzdHVzZXIifQ.r69ukkZ4cC8WzWcwpIl6FM5ITHmOatbx9GDVjFSilUdWz4e2f7DdKt8esRmQNDKfljhQmDCnarT

#### Create IAM Role for the Agents

In [27]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_FNOL_ROLE_NAME

execution_role_arn_fnol = create_agentcore_runtime_execution_role(AWS_FNOL_ROLE_NAME)

ℹ️ Role AWSFNOLAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1


In [28]:
cognito_config.get("client_id")

'ftjv7dm96qf84cc4n245krvt8'

In [29]:
cognito_config.get("discovery_url")

'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_JkqGFEXTz/.well-known/openid-configuration'

In [30]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_fnol_agent = Runtime()
fnol_agent_name="validation_assistant"

region = boto_session.region_name

# Configure the deployment
response_fnol_agent = agentcore_runtime_fnol_agent.configure(
    # entrypoint="agents/claimvalidation-agent.py",
    entrypoint="agents/claimvalidation-agent-2.py",
    # entrypoint="agents/fnol-agent-2.py",
    execution_role=execution_role_arn_fnol,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=fnol_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_fnol_agent)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/claimvalidation-agent-2.py, bedrock_agentcore_name=claimvalidation-agent-2
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/claimvalidation-agent-2.py, bedrock_agentcore_name=claimvalidation-agent-2
Memory configured with STM only
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Memory configured with STM only
Configuring BedrockAgentCore agent: validation_assistant
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: validation_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Memory configuration: Short-term memory only
Found existing memory ID from previous launch: validation_assistant_mem-ANSzlKEKLF
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Found existing memory ID from previous launch: validation_assistant_mem-ANSzlKEKLF


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Keeping 'validation_assistant' as default agent
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.config:Keeping 'validation_assistant' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [31]:
launch_result_fnol= agentcore_runtime_fnol_agent.launch()
print("Launch completed:", launch_result_fnol.agent_arn)

fnol_agent_arn = launch_result_fnol.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • No local Docker required
💡 Available deployment modes:
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentc

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-validation_assistant


Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-fc6456f075
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-fc6456f075
Using dockerignore.template with 45 patterns for zip filtering
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: validation_assistant/source.zip
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Uploaded source to S3: validation_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-validation_assistant-builder
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Updated CodeBuild project: bedrock-agentcore-validation_assistant-builder
Starting CodeBuild build (this may take several minutes)...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/validation_assistant-BpVgjG6ICV


In [32]:
status_response = agentcore_runtime_fnol_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
INFO:bedrock_agentcore_starter_toolkit.operations.memory.manager:✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: validation_assistant_mem-ANSzlKEKLF...
INFO:bedrock_agentcore_starter_toolkit.operations.memory.manager:🔎 Retrieving memory resource with ID: validation_assistant_mem-ANSzlKEKLF...
  Found memory: validation_assistant_mem-ANSzlKEKLF
INFO:bedrock_agentcore_starter_toolkit.operations.memory.manager:  Found memory: validation_assistant_mem-ANSzlKEKLF
Retrieved Bedrock AgentCore status for: validation_assistant
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: validation_assistant


Final status: READY


### Export and save outputs
Export variables to be used in next notebooks:

In [33]:
FNOL_AGENT_ID = launch_result_fnol.agent_id
FNOL_AGENT_ARN = launch_result_fnol.agent_arn
FNOL_AGENT_NAME = fnol_agent_name


FNOL_COGNITO_CLIENT_ID = cognito_config.get("client_id")
FNOL_COGNITO_SECRET = cognito_config.get("client_secret")
FNOL_DISCOVERY_URL = cognito_config.get("discovery_url")

%store FNOL_AGENT_ID
%store FNOL_AGENT_ARN
%store FNOL_AGENT_NAME
%store FNOL_COGNITO_CLIENT_ID
%store FNOL_COGNITO_SECRET
%store FNOL_DISCOVERY_URL

Stored 'FNOL_AGENT_ID' (str)
Stored 'FNOL_AGENT_ARN' (str)
Stored 'FNOL_AGENT_NAME' (str)
Stored 'FNOL_COGNITO_CLIENT_ID' (str)
Stored 'FNOL_COGNITO_SECRET' (str)
Stored 'FNOL_DISCOVERY_URL' (str)


In [34]:
FNOL_COGNITO_CLIENT_ID

'ftjv7dm96qf84cc4n245krvt8'

In [35]:
cognito_config.get("client_id")

'ftjv7dm96qf84cc4n245krvt8'

In [36]:
cognito_config.get("discovery_url")

'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_JkqGFEXTz/.well-known/openid-configuration'

In [37]:
from helpers.utils import put_ssm_parameter, SSM_FNOL_AGENT_ARN

put_ssm_parameter(SSM_FNOL_AGENT_ARN, FNOL_AGENT_ARN)


In [38]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [39]:
bearer_token

'eyJraWQiOiJsdnVjTXJjNmpqOXI2Yit2RlpnNks1WnZxMkF3ZjhPRklrXC9cLys5Zk8xUlU9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiIxNDI4MTQ2OC0xMDQxLTcwNWYtYmNhZi05MmZiMzNjY2EzMzUiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9Ka3FHRkVYVHoiLCJjbGllbnRfaWQiOiJmdGp2N2RtOTZxZjg0Y2M0bjI0NWtydnQ4Iiwib3JpZ2luX2p0aSI6IjBjNDU5NjQ5LTg2OWMtNDY3NC1iZTZjLWYzMzFkMDA4NjlhMiIsImV2ZW50X2lkIjoiMDExMDAyOWMtZDVmOC00MTFiLWE3MDMtZGIyNmI1OGNiODhkIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJhd3MuY29nbml0by5zaWduaW4udXNlci5hZG1pbiIsImF1dGhfdGltZSI6MTc2OTEzMDg3OSwiZXhwIjoxNzY5MTM0NDc5LCJpYXQiOjE3NjkxMzA4NzksImp0aSI6IjU3NThhMDhkLTY5MzEtNDdjMi1iOTNiLWZiZWFlMDYwNmQ1ZiIsInVzZXJuYW1lIjoidGVzdHVzZXIifQ.O1uI6IQw_BXeYzBgPBpv7nWVYDuu_snkYME-U5ySfi-xr-Oz_uSCQAB1BcQlRC5sDHRTc9FOZyKn_Pc8S7FzZP0n-Qz3qsn0BC6y-N14AV3cCISMAHTslqLtlCB39PnIRn-nriZmZ5kcs_ioK8aYLdgA9zIHu3gpfxKn_A5k7kD0YJJJZhW8AiluO4I3FzZbbpsZFyoPmJYmTeL_M7eTHy-IQEPZ2v_GsiFMzPZIFyiTdAcm_ZPDPTKhPGtgEDcYx1lNVsodW5x9k6fJ74E1JwHaTtPuIY2fdarIlVLFEbzW11qiy

In [40]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [41]:
fetch_agent_card(fnol_agent_arn)

ERROR:__main__:Error fetching agent card: 424 Client Error: Failed Dependency for url: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A161615149547%3Aruntime%2Fvalidation_assistant-BpVgjG6ICV/invocations/.well-known/agent-card.json


### Test agents
Now, let's invoke the first agent, using A2A:

In [ ]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

## 7. Process FNOL Data

In [ ]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

print("✓ FNOL data loaded successfully")
print(f"\nClaim Number: {fnol_data['fnol']['claimNumber']}")
print(f"Report Date: {fnol_data['fnol']['reportDate']}")
print(f"Policyholder: {fnol_data['policyholder']['firstName']} {fnol_data['policyholder']['lastName']}")
print(f"Vehicle: {fnol_data['vehicle']['insuredVehicle']['year']} {fnol_data['vehicle']['insuredVehicle']['make']} {fnol_data['vehicle']['insuredVehicle']['model']}")

In [ ]:
# Create processing prompt
fnol_prompt = f"""
Please process the following FNOL (First Notice of Loss) data.

FNOL Data:
{json.dumps(fnol_data, indent=2)}
"""

print("=" * 80)
print("PROCESSING FNOL DATA")
print("=" * 80)
print("\n FNOL prompt is ready ...\n")

In [ ]:
result = await send_sync_message(fnol_agent_arn, fnol_prompt)
formatted_output = format_agent_response(result)
print(formatted_output)